In [ ]:
# Iniciando Componentes

import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\sql_assistant_v3")
from src.components.memory.memory import Memory
from src.components.collector.collector import AppDataCollector, LLMResponseCollector
from src.components.memory.memory import Memory
from src.components.memory.memory_interfaces import AIMessage, HumanMessage
from typing import List, Union

# Memoria
messages: List[Union[AIMessage, HumanMessage]]  = []
memory = Memory(messages)

In [7]:
# Ejecutando
from src.app.app import chat


current_conversation_summary = """The conversation starts with the user introducing themselves as "Lauther"."""
current_slots = "None"
# ai_response = None

# user_message = "hi there, my name is Lauther"
user_message = "could you give me a list of measurement systems from database?"
# user_message = "now I would like the max temperature for the second one in list"
# user_message = "Could you tell me the date when it was measured?"
# user_message = "now I want the static pressure please"
# user_message = "and what about the max static pressure?"
# user_message = "and when it was measured?"


res = chat(
    user_message=user_message,
    chat_document="", #! Hay que agregar el chat document, deberia venir de una busqueda a la base de datos
)

generate-request
{'text': '\nsummary:  Lauther introduces themselves, then asks the assistant for a list of measurement systems from the database, which the assistant responds to.\nuser_intent: The user wants to get a list of measurement systems from the database.\nslots: database=measurement_systems'} 


enhanced-request
{'text': '\nresponse:  The user is looking for a list of measurement systems from the measurement systems database.'} 


request-type
{'text': '\ntype:  complex\nanalysis: This input is out of my knowledge, this type of request is related to getting information from the measurement systems database, which I do not have access to.'} 


complex_request_process_modification
enhanced-request
{'text': '\nresponse:  The user is seeking a list of measurement systems from the measurement_systems database.'} 


Se esta levantando el modelo de embeddings :D, listo en: 44.79123306274414s
technical-terms
{'text': '\nterms: measurement systems, measurement_systems'} 


flavored-re

C:\Users\lauth\OneDrive\Desktop\sql_assistant_v3\src\utils\sql_utils.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


summary-response-sql
{'text': 'response: Here is a list of measurement systems: EMED-3138.11-128, EMED-3138.12-050, EMED-3138.12-052, EMED-3138.12-065, EMED-3138.12-105, EST-3138.00-CONS_UDM, EST-3138.12-PILOTO, EST-LIBTQ-CX, FQIT-3138.12-015, TANQ-3138.11-011.'} 


post-process-translation
{'text': 'detected_language: English.  \nresponse: Here is a list of measurement systems: EMED-3138.11-128, EMED-3138.12-050, EMED-3138.12-052, EMED-3138.12-065, EMED-3138.12-105, EST-3138.00-CONS_UDM, EST-3138.12-PILOTO, EST-LIBTQ-CX, FQIT-3138.12-015, TANQ-3138.11-011.'} 




In [6]:
# Limpiando los recolectores
collector = AppDataCollector()
llm_collector = LLMResponseCollector()

In [ ]:
print(collector.current_conversation_data)

In [4]:
ai_response = collector.current_conversation_data.last_ai_message
print(ai_response)

AIMessage(
  replied_user_message_id=UUID('197f400e-232a-4418-a85a-1e8f8c2be7ee')
  message_id=UUID('9097af7a-82fb-485b-acd2-e1e74ec01f69'),
  message='Hi Lauther! Nice to meet you! Is there something I can help you with or would you like to know more about the measurement systems I have information on?',
  message_type='AI',
  date_created=datetime.datetime(2024, 7, 22, 18, 19, 30, 523907),
  sql_response=None,
  dataframe=None,
)


In [ ]:
print(len(llm_collector.llm_responses))
print(llm_collector.llm_responses[7].prompt)


In [8]:
print(memory.list_chat_messages())


<Human>: 2024-07-22 18:19:20.707579
hi there, my name is Lauther
<Assistant>: 2024-07-22 18:19:30.523907
Hi Lauther! Nice to meet you! Is there something I can help you with or would you like to know more about the measurement systems I have information on?
<Human>: 2024-07-22 18:21:04.919770
could you give me a list of measurement systems from database?
<Assistant>: 2024-07-22 18:22:28.578157
Here is a list of measurement systems: EMED-3138.11-128, EMED-3138.12-050, EMED-3138.12-052, EMED-3138.12-065, EMED-3138.12-105, EST-3138.00-CONS_UDM, EST-3138.12-PILOTO, EST-LIBTQ-CX, FQIT-3138.12-015, TANQ-3138.11-011.
Here is a dataframe from SQL: 
|    | Measurement_System_Name   | Measurement_System_Tag   |
|---:|:--------------------------|:-------------------------|
|  0 | EMED-3138.11-128          | EMED-3138.11-128         |
|  1 | EMED-3138.12-050          | EMED-3138.12-050         |
|  2 | EMED-3138.12-052          | EMED-3138.12-052         |
|  3 | EMED-3138.12-065          | EMED-

In [ ]:
collector.terms_dictionary

In [ ]:
import json
print(json.dumps(collector.terms_dictionary, indent=2))

In [ ]:
import sqlglot
import pyperclip

sql = """SELECT var.Valor AS 'Static Pressure (kPa)', med.Tag AS 'Measurement System Tag' FROM dbo_v2.var_variable_datos AS var INNER JOIN dbo_v2.med_sistema_medicion AS med ON var.idSistemaMedicion_fk = med.Id WHERE med.Tag = 'EMED-3138.12-050' AND var.Fecha = '2023-04-21' AND var.IdVariable_fk = 12 AND EXISTS (SELECT 1 FROM dbo_v2.var_variable_datos AS temp WHERE temp.idSistemaMedicion_fk = med.Id AND temp.Fecha = '2023-04-21' AND temp.IdVariable_fk = 6 AND temp.Valor <= 33.354)"""
code = sqlglot.transpile(sql, write="tsql", identify=True, pretty=True)[0]
pyperclip.copy(code)
print(code)